In [11]:
import yfinance as yf
import statsmodels.api as sm
import pandas as pd

In [2]:
aapl = yf.Ticker("AAPL")
data = aapl.history(start = "2018-01-01")
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-01-02 00:00:00-05:00,40.175162,40.680421,39.962668,40.670975,102223600,0.0,0.0
2018-01-03 00:00:00-05:00,40.734730,41.211658,40.600153,40.663898,118071600,0.0,0.0
2018-01-04 00:00:00-05:00,40.737092,40.956669,40.628487,40.852783,89738400,0.0,0.0
2018-01-05 00:00:00-05:00,40.949575,41.405251,40.857495,41.317894,94640000,0.0,0.0
2018-01-08 00:00:00-05:00,41.164429,41.461916,41.065263,41.164429,82271200,0.0,0.0


In [4]:
data['Return'] = data['Close'].pct_change()
data.dropna(inplace=True)

In [7]:
# Add a column for the target variable (Next days Close/Return)
data['Target Close'] = data['Close'].shift(-1)
data['Target Return'] = data['Return'].shift(-1)
data.dropna(inplace=True)

In [8]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Return,Target Close,Target Return
Date,,,,,,,,,,
2018-01-03 00:00:00-05:00,40.734730,41.211658,40.600153,40.663898,118071600,0.0,0.0,-0.000174,40.852783,0.004645
2018-01-04 00:00:00-05:00,40.737092,40.956669,40.628487,40.852783,89738400,0.0,0.0,0.004645,41.317894,0.011385
2018-01-05 00:00:00-05:00,40.949575,41.405251,40.857495,41.317894,94640000,0.0,0.0,0.011385,41.164429,-0.003714
2018-01-08 00:00:00-05:00,41.164429,41.461916,41.065263,41.164429,82271200,0.0,0.0,-0.003714,41.159721,-0.000114
2018-01-09 00:00:00-05:00,41.211664,41.332075,40.942508,41.159721,86336000,0.0,0.0,-0.000114,41.150269,-0.000230
...,...,...,...,...,...,...,...,...,...,...
2024-03-28 00:00:00-04:00,171.750000,172.229996,170.509995,171.479996,65672700,0.0,0.0,-0.010559,170.029999,-0.008456
2024-04-01 00:00:00-04:00,171.190002,171.250000,169.479996,170.029999,46240500,0.0,0.0,-0.008456,168.839996,-0.006999
2024-04-02 00:00:00-04:00,169.080002,169.339996,168.229996,168.839996,49329500,0.0,0.0,-0.006999,169.649994,0.004797


# Close Price Regression

In [12]:
# Rgression model to predict the next days Close
model = sm.regression.linear_model.OLS(data['Target Close'],
                                       sm.tools.add_constant(data['Close']))
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Target Close   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 8.950e+05
Date:                Sun, 07 Apr 2024   Prob (F-statistic):               0.00
Time:                        00:03:06   Log-Likelihood:                -3463.8
No. Observations:                1573   AIC:                             6932.
Df Residuals:                    1571   BIC:                             6942.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2022      0.130      1.552      0.121      -0.053       0.458
Close          0.9989      0.001    946.034      0.000       0.997       1.001
==============================================================================
Omnibus:                      129.380   Durbin-Watson:                   2.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              738.645
Skew:                          -0.049   Prob(JB):                    4.03e-161
Kurtosis:                       6.356   Cond. No.                         291.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The above model gave very high r-squared value.

# Regression on returns

In [13]:
# Rgression model to predict the next days Close
model = sm.regression.linear_model.OLS(data['Target Return'],
                                       sm.tools.add_constant(data['Return']))
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Target Return   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     14.93
Date:                Sun, 07 Apr 2024   Prob (F-statistic):           0.000116
Time:                        00:05:26   Log-Likelihood:                 3951.7
No. Observations:                1573   AIC:                            -7899.
Df Residuals:                    1571   BIC:                            -7889.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0012      0.000      2.439      0.015       0.000       0.002
Return        -0.0970      0.025     -3.865      0.000      -0.146      -0.048
==============================================================================
Omnibus:                      165.029   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1289.499
Skew:                          -0.069   Prob(JB):                    9.75e-281
Kurtosis:                       7.433   Cond. No.                         50.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The above model gave very 0 value.